# Initialization, configure API host and key, and create new API instance

In [7]:
var NucleusApi = require('nucleus_api');
var fs = require('fs'); 
var csvParser = require('csv-parse');

var host = 'UPDATE-WITH-API-HOST';
var apiKey = 'UPDATE-WITH-API-KEY';

// Create API instance
var apiClient = new NucleusApi.ApiClient(host, apiKey);
var apiInstance = new NucleusApi.NucleusApi(apiClient);

# Dataset APIs

## Append file from local drive to dataset

In [2]:
console.log("--------- Append file from local drive to dataset -----------");
var file = "quarles20181109a.pdf"; // File | 
var dataset = "dataset_test"; // String | Destination dataset where the file will be inserted.
var opts = { 
    //'metadata': "" // String | Optional json containing additional document metadata. Eg: {\"time\":\"01/01/2001\",\"author\":\"me\"}
};

var fileStream = fs.createReadStream(file)

var callback = function(error, data, response) {
    if (error) {
        console.error(error);
    } else {
        console.log(data.result + ' added to dataset ' + dataset);
        console.log('-------------------------------------------------------------');
    }
};
apiInstance.postUploadFile(fileStream, dataset, opts, callback);
console.log('-------------------------------------------------------------');


--------- Append file from local drive to dataset -----------
-------------------------------------------------------------
quarles20181109a.pdf added to dataset dataset_test
-------------------------------------------------------------


## Append file from URL to dataset

In [3]:
console.log("------------ Append file from URL to dataset ---------------");

var file_url = "https://www.federalreserve.gov/newsevents/speech/files/quarles20181109a.pdf"
var dataset = "dataset_test"
var payload = new NucleusApi.UploadURLModel(dataset=dataset,
                 file_url=file_url); // UploadURLModel | 

var callback = function(error, data, response) {
    if (error) {
        console.error(error);
    } else {
        console.log(data.result + ' added to dataset ' + dataset);
        console.log('-------------------------------------------------------------');
    }
};
apiInstance.postUploadUrl(payload, callback);

console.log('-------------------------------------------------------------');

------------ Append file from URL to dataset ---------------
-------------------------------------------------------------
https://www.federalreserve.gov/newsevents/speech/files/quarles20181109a.pdf added to dataset dataset_test
-------------------------------------------------------------


## Append json from csv to dataset

In [41]:
// This dataset will be used to test all topics and documents APIs
console.log("----------- Append json from CSV to dataset -----------------");

var csvFile = "trump-tweets-100.csv"
var dataset = "trump_tweets"

var callback = function(error, data, response) {
    if (error) {
        console.error(error);
    } else {
        //console.log('API called successfully. Returned data: ' + data.result);
    }
};

var processCsvData = function(data, rows, dataset) {
    console.log('Processing CSV data...');
    // The first row is header. Skip it.
    for (var i = 1; i < rows; i++) {
        var curRow = data[i];
        var payload = {
            'dataset': dataset,
            //'language': 'english', // optional.
            'document': {'time'   : curRow[2],
                         'title'  : curRow[4],
                         'content': curRow[1],
                         'source' : curRow[0],
                         'author' : curRow[3]}};
        
        apiInstance.postAppendJsonToDataset(payload, callback);
    }
    console.log(rows-1 + ' documents added to dataset ' + dataset);
    console.log('-------------------------------------------------------------')
}

var csvData=[];
fs.createReadStream(csvFile)
    .pipe(csvParser())
    .on('data', function(csvrow) {
        csvData.push(csvrow);        
    })
    .on('end',function() {
        processCsvData(csvData, csvData.length, dataset);
    });

console.log('-------------------------------------------------------------')

----------- Append json from CSV to dataset -----------------
-------------------------------------------------------------
Processing CSV data...
100 documents added to dataset trump_tweets


## List available datasets

In [42]:
console.log('---------------- List available datasets ---------------------');

var callback = function(error, data, response) {
    if (error) {
        console.error(error);
    } else {
        var listDatasets = data.result;
        var datasetCount = listDatasets.length;
        console.log(datasetCount + ' datasets in the database:')
        for (var i = 0; i < datasetCount; i++) {
            console.log('    ' + listDatasets[i]);
        }
        console.log('-------------------------------------------------------------')
    }
};
apiInstance.getListDatasets(callback);

console.log('-------------------------------------------------------------');

---------------- List available datasets ---------------------
-------------------------------------------------------------
3 datasets in the database:
    dataset_test
    trump_tweets
    trump_tweets_full
-------------------------------------------------------------


## Get dataset information

In [43]:
console.log('--------------- Get dataset information -------------------');
var dataset = 'dataset_test'; // String | Dataset name.
var opts = { 
    'query': '', // String | Fulltext query, using mysql MATCH boolean query format.
    'metadataSelection': '', // String | json object of {\"metadata_field\":[\"selected_values\"]}
    'timePeriod': '' // String | Time period selection
};

var callback = function(error, data, response) {
    if (error) {
        console.error(error);
    } else {
        console.log('Information about dataset ' + dataset);
        console.log('    Language: ' + data.result.detected_language)
        console.log('    Number of documents: ' + data.result.num_documents);
        console.log('    Time range: ' + data.result.time_range[0] +
              ' to ' + data.result.time_range[1]);
        console.log('-------------------------------------------------------------');
    }
};
apiInstance.getDatasetInfo(dataset, opts, callback);

console.log('-------------------------------------------------------------');

--------------- Get dataset information -------------------
-------------------------------------------------------------
Information about dataset dataset_test
    Language: en
    Number of documents: 200
    Time range: 1534063860 to 1534646340
-------------------------------------------------------------


## Delete document

In [44]:
console.log('--------------------- Delete document -----------------------');
var dataset = 'dataset_test'
var docid = '1'
var payload = new NucleusApi.Deletedocumentmodel(
    dataset=dataset,
    docid=docid); // Deletedocumentmodel | 


var callback = function(error, data, response) {
    if (error) {
        console.error(error);
    } else {
        console.log(data.result);
        console.log('-------------------------------------------------------------');
    }
};
apiInstance.postDeleteDocument(payload, callback);

console.log('-------------------------------------------------------------');

--------------------- Delete document -----------------------
-------------------------------------------------------------
Document deleted
-------------------------------------------------------------


## Delete dataset

In [45]:
console.log('--------------------- Delete dataset ------------------------');
var dataset = 'dataset_test'
var payload = new NucleusApi.Deletedatasetmodel(
    dataset=dataset); // Deletedatasetmodel | 

var callback = function(error, data, response) {
    if (error) {
        console.error(error);
    } else {
        console.log(dataset + ' ' + data.result);
    }
};
apiInstance.postDeleteDataset(payload, callback);


console.log('-------------------------------------------------------------');

--------------------- Delete dataset ------------------------
-------------------------------------------------------------
dataset_test Dataset deleted


# Topic APIs

## Get list of topics from dataset

In [46]:
console.log('------------- Get list of topics from dataset --------------');
var dataset = "trump_tweets"; // String | Dataset name.

var opts = { 
    'query': '', // String | Fulltext query, using mysql MATCH boolean query format. Example, (\"word1\" OR \"word2\") AND (\"word3\" OR \"word4\")
    //'customStopWords': "\["real", "hillary"\]", // String | List of stop words.
    'numTopics': 8, // Number | Number of topics to be extracted from the dataset.
    'numKeywords': 8, // Number | Number of keywords per topic that is extracted from the dataset.
    'metadataSelection': '', // String | json object of {\"metadata_field\":[\"selected_values\"]}
    'timePeriod': '' // String | Time period selection
    //'excludedDocs': '' // String | List of document IDs that should be excluded from the analysis. Example, \"docid1, docid2, ..., docidN\" 
};

var callback = function(error, data, response) {
    if (error) {
        console.error(error);
    } else {
        for (var i = 0; i < data.result.length; i++) {
            var res = data.result[i];
            console.log('Topic', i, 'keywords:');
            console.log('    Keywords:', res.topic);
            console.log('    Keyword weights:',  res.keywords_weight);
            console.log('    Strength:', res.strength);
            var doc_topic_exposure_sel = [];  // list of non-zero doc_topic_exposure
            var doc_id_sel = [];    // list of doc ids matching doc_topic_exposure_sel
            for (var j = 0; j < res.doc_topic_exposure.length; j++) {
                var doc_topic_exp = parseFloat(res.doc_topic_exposure[j]);
                if (doc_topic_exp != 0) {
                    doc_topic_exposure_sel.push(doc_topic_exp)
                    doc_id_sel.push(res.doc_id[j])
                }
            }
            console.log('    Document IDs:', doc_id_sel)
            console.log('    Document exposures:', doc_topic_exposure_sel)

            console.log('---------------')
        }
    }
};
apiInstance.getTopicApi(dataset, opts, callback);

console.log('-------------------------------------------------------------');

------------- Get list of topics from dataset --------------
-------------------------------------------------------------
Topic 0 keywords:
    Keywords: illegal aliens;united permission;permission illegal;enter united;aliens treated;aliens illegal;treated entered;chuck schumer
    Keyword weights: [ '0.06397001809253038',
  '0.1245799948307056',
  '0.1245799948307056',
  '0.1245799948307056',
  '0.2491599896614112',
  '0.1245799948307056',
  '0.1245799948307056',
  '0.06397001809253038' ]
    Strength: 0.17732008399576646
    Document IDs: [ '57', '63', '3657', '3663' ]
    Document exposures: [ 0.24361226030827163,
  0.23113677211743835,
  0.26990959957549243,
  0.2553413679987976 ]
---------------
Topic 1 keywords:
    Keywords: bruce ohr;ohr justice;justice department;wife nelly;fusion gps;christopher steele;department believe;believe accused
    Keyword weights: [ '0.20881568306864162',
  '0.1061422989971866',
  '0.143683553601218',
  '0.143683553601218',
  '0.12069616168279465',

## Get topic summary

In [47]:
console.log('------------------- Get topic summary -----------------------');
var dataset = "trump_tweets"; // String | Dataset name.

var opts = { 
    'query': "", // String | Fulltext query, using mysql MATCH boolean query format. Example, (\"word1\" OR \"word2\") AND (\"word3\" OR \"word4\")
    //'customStopWords': "customStopWords_example", // String | List of stop words
    'numTopics': 8, // Number | Number of topics to be extracted from the dataset and summarized.
    'numKeywords': 8, // Number | Number of keywords per topic that is extracted from the dataset.
    'metadataSelection': "", // String | json object of {\"metadata_field\":[\"selected_values\"]}
    'timePeriod': "", // String | Time period selection
    'summaryLength': 6, // Number | The maximum number of bullet points a user wants to see in each topic summary.
    'contextAmount': 0, // Number | The number of sentences surrounding key summary sentences in the documents that they come from.
    'numDocs': 20 // Number | The maximum number of key documents to use for summarization.
    //'excludedDocs': "excludedDocs_example" // String | List of document IDs that should be excluded from the analysis. Example, \"docid1, docid2, ..., docidN\" 
};

var callback = function(error, data, response) {
    if (error) {
        console.error(error);
    } else {
        for (var i = 0; i < data.result.length; i++) {
            var res = data.result[i];
            console.log('Topic', i, 'summary:')
            console.log('    Keywords:', res.topic)
            for (var j = 0; j < res.summary.length; j++) {
                var summary = res.summary[j];
                console.log('    Document ID: ' + summary.sourceid);
                console.log('        Title: ' + summary.title);
                console.log('        Sentences: ' + summary.sentences);
                console.log('        Author: ' + summary.attribute.author);
                console.log('        Source: ' + summary.attribute.source);
                var docTimestamp = parseFloat(summary.attribute.time);
                var docTime = new Date(docTimestamp * 1000);
                console.log('        Time: ' + docTime.getFullYear() + '-' +
                                               docTime.getMonth() + '-' +
                                               docTime.getDate());
                console.log('---------------');
            }
        }
        console.log('-------------------------------------------------------------');
    }
};
apiInstance.getTopicSummaryApi(dataset, opts, callback);

console.log('-------------------------------------------------------------');

------------------- Get topic summary -----------------------
-------------------------------------------------------------
Topic 0 summary:
    Keywords: illegal aliens;united permission;permission illegal;enter united;aliens treated;aliens illegal;treated entered;chuck schumer
    Document ID: 57
        Title: D_Trump2018_8_15_13_18
        Sentences: “People who enter the United States without our permission are illegal aliens and illegal aliens should not be treated the same as people who entered the U.S. legally.” Chuck Schumer in 2009 before he went left and haywire!
        Author: D_Trump57
        Source: null
        Time: 2018-7-15
---------------
    Document ID: 63
        Title: D_Trump2018_8_15_12_44
        Sentences: “People who enter the United States without our permission are illegal aliens and illegal aliens should not be treated the same as people who enters the U.S. legally.”  Chuck Schumer in 2009 before he went left and haywire!
        Author: D_Trump63
     

        Author: D_Trump29
        Source: null
        Time: 2018-7-17
---------------
    Document ID: 30
        Title: D_Trump2018_8_17_0_45
        Sentences: Then followed by “Richard Blumenthal of Connecticut is a FAKE War Hero...” So true a total Fake!
        Author: D_Trump30
        Source: null
        Time: 2018-7-17
---------------
    Document ID: 3630
        Title: D_Trump2018_8_17_0_45
        Sentences: Then followed by “Richard Blumenthal of Connecticut is a FAKE War Hero..”.
        Author: D_Trump30
        Source: null
        Time: 2018-7-17
---------------
Topic 6 summary:
    Keywords: fake news;news media;wacky omarosa;presidential lowlife;omarosa modern;modern form;lowlife omarosa;form communication
    Document ID: 93
        Title: D_Trump2018_8_13_14_21
        Sentences: While I know it’s “not presidential” to take on a lowlife like Omarosa and while I would rather not be doing so this is a modern day form of communication and I know the Fake News Media w

## Get topic sentiment

In [48]:
console.log('---------------- Get topic sentiment ------------------------');
var dataset = "trump_tweets"; // String | Dataset name.
var opts = { 
    'query': "", // String | Fulltext query, using mysql MATCH boolean query format. Example, (\"word1\" OR \"word2\") AND (\"word3\" OR \"word4\")
    //'customStopWords': "customStopWords_example", // String | List of stop words
    'numTopics': 8, // Number | Number of topics to be extracted from the dataset.
    'numKeywords': 8, // Number | Number of keywords per topic that is extracted from the dataset.
    'metadataSelection': "", // String | json object of {\"metadata_field\":[\"selected_values\"]}
    'timePeriod': "", // String | Time period selection
    //'excludedDocs': "excludedDocs_example", // String | List of document IDs that should be excluded from the analysis. Example, \"docid1, docid2, ..., docidN\" 
    //'customDictFile': "/path/to/file.txt" // File | Custom sentiment dictionary JSON file.
};

var callback = function(error, data, response) {
    if (error) {
        console.error(error);
    } else {
        for (var i = 0; i < data.result.length; i++) {
            var res = data.result[i];

            console.log('Topic', i, 'sentiment:');
            console.log('    Keywords:', res.topic);
            console.log('    Sentiment:', res.sentiment);
            console.log('    Strength:', res.strength);
            console.log('    Document IDs:', res.doc_id);
            console.log('    Document Sentiments:', res.doc_sentiment);
            console.log('    Document Scores:', res.doc_score);
    
            console.log('---------------');
        }
        console.log('-------------------------------------------------------------');
    }
};
apiInstance.postTopicSentimentApi(dataset, opts, callback);

console.log('-------------------------------------------------------------');

---------------- Get topic sentiment ------------------------
-------------------------------------------------------------
Topic 0 sentiment:
    Keywords: illegal aliens;united permission;permission illegal;enter united;aliens treated;aliens illegal;treated entered;chuck schumer
    Sentiment: -0.5109210311394632
    Strength: 0.17732008399576646
    Document IDs: [ '57', '63', '3657', '3663' ]
    Document Sentiments: [ '-0.46153846153846156',
  '-0.46153846153846156',
  '-0.5555555555555556',
  '-0.5555555555555556' ]
    Document Scores: [ '0.2436122603082716,0.23113677211743833,0.26990959957549243,0.2553413679987976' ]
---------------
Topic 1 sentiment:
    Keywords: bruce ohr;ohr justice;justice department;wife nelly;fusion gps;christopher steele;department believe;believe accused
    Sentiment: -0.2639182756585356
    Strength: 0.18478462540354465
    Document IDs: [ '11', '12', '13', '31', '33', '70', '3613', '3633', '3670' ]
    Document Sentiments: [ '-0.3333333333333333',
 

## Get topic consensus

In [49]:
console.log('---------------- Get topic consensus ------------------------')
var dataset = "trump_tweets"; // String | Dataset name.

var opts = { 
    'query': "", // String | Fulltext query, using mysql MATCH boolean query format. Example, (\"word1\" OR \"word2\") AND (\"word3\" OR \"word4\")
    //'customStopWords': "", // String | List of stop words
    'numTopics': 8, // Number | Number of topics to be extracted from the dataset.
    'numKeywords': 8, // Number | Number of keywords per topic that is extracted from the dataset.
    'metadataSelection': "", // String | json object of {\"metadata_field\":[\"selected_values\"]}
    'timePeriod': "", // String | Time period selection
    //'excludedDocs': "excludedDocs_example", // String | List of document IDs that should be excluded from the analysis. Example, \"docid1, docid2, ..., docidN\" 
    'customDictFile': "" // File | Custom sentiment dictionary JSON file.
};

var callback = function(error, data, response) {
    if (error) {
        console.error(error);
    } else {
        for (var i = 0; i < data.result.length; i++) {
            var res = data.result[i];
            console.log('Topic', i, 'consensus:');
            console.log('    Keywords: ' + res.topic);
            console.log('    Consensus: ' + res.consensus);
            console.log('    Strength: ' + res.strength);
        
            console.log('---------------');
        }
    }
};
apiInstance.postTopicConsensusApi(dataset, opts, callback);

console.log('-------------------------------------------------------------')

---------------- Get topic consensus ------------------------
-------------------------------------------------------------
Topic 0 consensus:
    Keywords: illegal aliens;united permission;permission illegal;enter united;aliens treated;aliens illegal;treated entered;chuck schumer
    Consensus: 1
    Strength: 0.17732008399576646
---------------
Topic 1 consensus:
    Keywords: bruce ohr;ohr justice;justice department;wife nelly;fusion gps;christopher steele;department believe;believe accused
    Consensus: 1.0000000000000002
    Strength: 0.18478462540354465
---------------
Topic 2 consensus:
    Keywords: peter strzok;fired fbi;agent peter;fbi agent;disgraced fired;nelly fusion;documents bruce;department wife
    Consensus: 1
    Strength: 0.12008695528718116
---------------
Topic 3 consensus:
    Keywords: andrew cuomo;total meltdown;taxed andrew;highest taxed;governor highest;cuomo total;believe governor;america great
    Consensus: 0.9441841918616392
    Strength: 0.1411812087174

## Get topic historical analysis

In [50]:
console.log('------------ Get topic historical analysis ----------------');

var dataset = "trump_tweets"; // String | Dataset name.
var timePeriod = "6M"; // String | Time period selection
var updatePeriod = "d"; // String | Frequency at which the historical anlaysis is performed

var opts = { 
    'query': "", // String | Fulltext query, using mysql MATCH boolean query format. Example, (\"word1\" OR \"word2\") AND (\"word3\" OR \"word4\")
    //'customStopWords': "", // String | List of stop words
    'numTopics': 8, // Number | Number of topics to be extracted from the dataset.
    'numKeywords': 8, // Number | Number of keywords per topic that is extracted from the dataset.
    'metadataSelection': "", // String | json object of {\"metadata_field\":[\"selected_values\"]}
    'incStep': 1, // Number | Number of increments of the udpate period in between two historical computations.
    //'excludedDocs': "excludedDocs_example", // String | List of document IDs that should be excluded from the analysis. Example, \"docid1, docid2, ..., docidN\" 
    'customDictFile': "" // File | Custom sentiment dictionary JSON file.
};

var callback = function(error, data, response) {
    if (error) {
        console.error(error);
    } else {
        console.log('Printing historical metrics data...')
        for (var i = 0; i < data.result.length; i++) {
            var res = data.result[i];
            console.log('Topic', i, res.topic);
            console.log('    Timestamps:', res.time_stamps);
            console.log('    Strength:', res.strength);
            console.log('    Consensus:', res.consensus);
            console.log('    Sentiment:', res.sentiment);
            console.log('----------------');
        }
        console.log('-------------------------------------------------------------');
    }
};
apiInstance.postTopicHistoricalAnalysisApi(dataset, timePeriod, updatePeriod, opts, callback);

console.log('-------------------------------------------------------------');

------------ Get topic historical analysis ----------------
-------------------------------------------------------------


## Get author connectivity

In [51]:
console.log('----------------- Get author connectivity -------------------');


var dataset = "trump_tweets"; // String | Dataset name.
var targetAuthor = "D_Trump16"; // String | Name of the author to be analyzed.
var timePeriod = "12M"; // String | Time period selection
var opts = { 
    'query': "", // String | Fulltext query, using mysql MATCH boolean query format. Subject covered by the author, on which to focus the analysis of connectivity. Example, (\"word1\" OR \"word2\") AND (\"word3\" OR \"word4\")
    //'customStopWords': [""], // String | List of words possibly used by the target author that are considered not information-bearing.
    'metadataSelection': "", // String | json object of {\"metadata_field\":[\"selected_values\"]}
    //'excludedDocs': [""] // String | List of document IDs that should be excluded from the analysis. Example, \"docid1, docid2, ..., docidN\" 
};

var callback = function(error, data, response) {
    if (error) {
        console.error(error);
    } else {
        var res = data.result;
        console.log('Mainstream connections:')
        for (var i = 0; i < res.mainstream_connection.length; i++) {
            var mc = res.mainstream_connection[i];
            console.log('    Topic:', mc.topic);
            console.log('    Authors:', mc.authors);
        }
        
        console.log('Niche connections:')
        for (var i = 0; i < res.niche_connection.length; i++) {
            var nc = res.niche_connection[i];
            console.log('    Topic:', nc.topic);
            console.log('    Authors:', nc.authors);
        }
        console.log('-------------------------------------------------------------');
  }
};
apiInstance.getAuthorConnectivityApi(dataset, targetAuthor, timePeriod, opts, callback);

console.log('-------------------------------------------------------------');

----------------- Get author connectivity -------------------
-------------------------------------------------------------
Mainstream connections:
    Topic: worse hightax;hightax andrew;andrew cuomo
    Authors: [ 'D_Trump44' ]
Niche connections:
    Topic: statement america;cuomo statement;america great
    Authors: [ 'D_Trump24' ]
-------------------------------------------------------------


## Get topic delta

In [52]:
console.log('------------------- Get topic deltas -----------------------');

var dataset = "trump_tweets"; // String | Dataset with a time series component. These docs should reference a universe of entities that overlaps through time.
var timeStartT0 = "2018-08-12 00:00:00"; // String | Start date for the start-of-period dataset. Format: \"YYYY-MM-DD HH:MM:SS\" 
var timeEndT0 = "2018-08-15 13:00:00"; // String | End date for the start-of-period dataset. Format: \"YYYY-MM-DD HH:MM:SS\" 
var timeStartT1 = "2018-08-16 00:00:00"; // String | Start date for the end-of-period dataset. Format: \"YYYY-MM-DD HH:MM:SS\" 
var timeEndT1 = "2018-08-19 00:00:00"; // String | End date for the end-of-period dataset. Format: \"YYYY-MM-DD HH:MM:SS\" 

var opts = { 
   'query': "", // String | Fulltext query, using mysql MATCH boolean query format. Example, (\"word1\" OR \"word2\") AND (\"word3\" OR \"word4\")
   //'customStopWords': "customStopWords_example", // String | List of stop words.
   'numTopics': 8, // Number | Number of topics to be extracted from the dataset.
   'numKeywords': 8, // Number | Number of keywords per topic that is extracted from the dataset.
   'metadataSelection': "", // String | json object of {\"metadata_field\":[\"selected_values\"]}
   //'excludedDocs': "excludedDocs_example" // String | List of document IDs that should be excluded from the analysis. Example, \"docid1, docid2, ..., docidN\" 
};

var callback = function(error, data, response) {
    if (error) {
        console.error(error);
    } else {
        for (var i = 0; i < data.result.length; i++) {
            var res = data.result[i];
            console.log('Topic', i, 'changes in exposure:');
            console.log('    Keywords:', res.topic);
            console.log('    Document ID:', res.doc_id_t0, res.doc_id_t1);
            console.log('    Per Source Change in Exposure:', res.doc_topic_exposure_delta);
            console.log('---------------');
        }
    }
};
apiInstance.getTopicDeltaApi(dataset, timeStartT0, timeEndT0, timeStartT1, timeEndT1, opts, callback);

console.log('-------------------------------------------------------------');
//

------------------- Get topic deltas -----------------------
-------------------------------------------------------------
Topic 0 changes in exposure:
    Keywords: donald trump;witch hunt;hillary clinton;frame donald;clinton frame;jarrett foxandfriends;gregg jarrett;foxandfriends real
    Document ID: [ '62',
  '63',
  '65',
  '66',
  '67',
  '68',
  '69',
  '70',
  '71',
  '72',
  '73',
  '74',
  '75',
  '76',
  '77',
  '78',
  '80',
  '81',
  '82',
  '83',
  '84',
  '86',
  '87',
  '88',
  '89',
  '90',
  '91',
  '92',
  '93',
  '94',
  '95',
  '96',
  '97',
  '98',
  '99',
  '3663',
  '3669',
  '3670',
  '3698' ] [ '2',
  '3',
  '4',
  '6',
  '7',
  '8',
  '9',
  '10',
  '11',
  '12',
  '13',
  '15',
  '16',
  '17',
  '18',
  '22',
  '23',
  '24',
  '25',
  '26',
  '27',
  '28',
  '29',
  '30',
  '31',
  '32',
  '33',
  '34',
  '37',
  '38',
  '39',
  '40',
  '41',
  '42',
  '43',
  '44',
  '45',
  '46',
  '47',
  '3613',
  '3630',
  '3632',
  '3633',
  '3644',
  '3647' ]
    Per 

    Per Source Change in Exposure: [ '0',
  '0',
  '0',
  '0',
  '-0.0007367096248316801',
  '-0.0020635771524205575',
  '-0.5287293765577651',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '-0.0006530232868586794',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '-0.467817313378124',
  '0',
  '0' ]
---------------
Topic 7 changes in exposure:
    Keywords: peter strzok;fired fbi;agent peter;strzok fraud;rigged investigation;investigation started;fraud rigged;fbi agent
    Document ID: [ '62',
  '63',
  '65',
  '66',
  '67',
  '68',
  '69',
  '70',
  '71',
  '72',
  '73',
  '74',
  '75',
  '76',
  '77',
  '78',
  '80',
  '81',
  '82',
  '83',
  '84',
  '86',
  '87',
  '88',
  '89',
  '90',
  '91',
  '92',
  '93',
  '94',
  '95',
  '96',
  '97',
  '98',
  '99',
  '3663',
  '3669',
  '3670',
  '3698' ] [ '2',
  '3',
  '4',
  '6',
  '7',
  '8',
  '9',
  '10',
  '11',
  '12',
  '13',
 

# Document APIs

## Get document information without content

In [53]:
console.log('----------------- Display document information --------------------');
var dataset = "trump_tweets"; // String | Dataset name.
var opts = { 
    'docTitles': "[ \"D_Trump2018_8_18_1_47\" ]", // String | The title of the documents on which info is requested. Example: [ \"title 1\", \"title 2\" ] 
    'docIds': "[ \"11\", \"12\", \"13\" ]" // String | The docid of the documents on which info is requested. Example:[ \"docid1, docid2\" ]
};

var callback = function(error, data, response) {
    if (error) {
        console.error(error);
    } else {
        for (var i = 0; i < data.result.length; i++) {
            var res = data.result[i];
            console.log('Document ID:', res.sourceid);
            console.log('    Title:', res.title);
            console.log('    Author:', res.attribute.author);
            console.log('    Source:', res.attribute.source);
            console.log('    Time:', res.attribute.time);

            console.log('---------------')
        }
    }
};
apiInstance.getDocInfo(dataset, opts, callback);

console.log('-------------------------------------------------------------');

----------------- Display document information --------------------
-------------------------------------------------------------
Document ID: 10
    Title: D_Trump2018_8_18_1_47
    Author: D_Trump10
    Source: null
    Time: 1534582020
---------------
Document ID: 11
    Title: D_Trump2018_8_18_1_46
    Author: D_Trump11
    Source: null
    Time: 1534581960
---------------
Document ID: 12
    Title: D_Trump2018_8_18_1_37
    Author: D_Trump12
    Source: null
    Time: 1534581420
---------------
Document ID: 13
    Title: D_Trump2018_8_17_22_29
    Author: D_Trump13
    Source: null
    Time: 1534570140
---------------
Printing historical metrics data...
Topic 0 illegal aliens;united permission;permission illegal;enter united;aliens treated;aliens illegal;treated entered;chuck schumer
    Timestamps: [ '2018-07-09',
  '2018-07-10',
  '2018-07-11',
  '2018-07-12',
  '2018-07-13',
  '2018-07-14',
  '2018-07-15',
  '2018-07-16',
  '2018-07-17',
  '2018-07-18',
  '2018-07-19',
  '2018-

    Consensus: [ 'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  ... 81 more items ]
    Sentiment: [ 'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  '

    Strength: [ 'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  '0.130079567537263',
  '0.1444193020884498',
  '0.14118120871745254',
  '0.14118120871745254',
  '0.14118120871745254',
  '0.14118120871745254',
  '0.14118120871745254',
  '0.14118120871745254',
  '0.14118120871745254',
  '0.14118120871745254',
  '0.14118120871745254',
  '0.14118120871745254',
  '0.14118120871745254',
  '0.14118120871745254',
  '0.14118120871745254',
  '0.14288947642594674',
  '0.15521872142353502',
  '0.1966935989954026',
  '0.30737162678757335',
  '0.19566859133106917',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'n

----------------
Topic 5 war hero;richard blumenthal;blumenthal connecticut;fake war;connecticut fake;true total;total fake;hero true
    Timestamps: [ '2018-07-09',
  '2018-07-10',
  '2018-07-11',
  '2018-07-12',
  '2018-07-13',
  '2018-07-14',
  '2018-07-15',
  '2018-07-16',
  '2018-07-17',
  '2018-07-18',
  '2018-07-19',
  '2018-07-20',
  '2018-07-21',
  '2018-07-22',
  '2018-07-23',
  '2018-07-24',
  '2018-07-25',
  '2018-07-26',
  '2018-07-27',
  '2018-07-28',
  '2018-07-29',
  '2018-07-30',
  '2018-07-31',
  '2018-08-01',
  '2018-08-02',
  '2018-08-03',
  '2018-08-04',
  '2018-08-05',
  '2018-08-06',
  '2018-08-07',
  '2018-08-08',
  '2018-08-09',
  '2018-08-10',
  '2018-08-11',
  '2018-08-12',
  '2018-08-13',
  '2018-08-14',
  '2018-08-15',
  '2018-08-16',
  '2018-08-17',
  '2018-08-18',
  '2018-08-19',
  '2018-08-20',
  '2018-08-21',
  '2018-08-22',
  '2018-08-23',
  '2018-08-24',
  '2018-08-25',
  '2018-08-26',
  '2018-08-27',
  '2018-08-28',
  '2018-08-29',
  '2018-08-30',
  

    Consensus: [ 'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  '0.6820189238779336',
  '0.6820189238779336',
  '0.6820189238779336',
  '0.5678012054769198',
  '0.5678012054769198',
  '0.6016155982243083',
  '0.6016155982243083',
  '0.6016155982243083',
  '0.6016155982243083',
  '0.6016155982243083',
  '0.6016155982243083',
  '0.6016155982243083',
  '0.6016155982243083',
  '0.6016155982243083',
  '0.6016155982243083',
  '0.6016155982243083',
  '0.6016155982243083',
  '0.6016155982243083',
  '0.6016155982243083',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan'

## Display document details

In [54]:
console.log('----------------- Display document details --------------------');

var dataset = "trump_tweets"; // String | Dataset name.

var opts = { 
    'docTitles': "[ \"D_Trump2018_8_18_1_47\" ]", // String | The title of the documents on which info is requested. Example: [ \"title 1\", \"title 2\" ] 
    'docIds': "[ \"11\", \"12\", \"13\" ]" // String | The docid of the documents on which info is requested. Example:[ \"docid1, docid2\" ]
};

var callback = function(error, data, response) {
    if (error) {
        console.error(error);
    } else {
        for (var i = 0; i < data.result.length; i++) {
            var res = data.result[i];
            console.log('Document ID:' + res.sourceid);
            console.log('    Title:' + res.title);
            console.log('    Author:' + res.attribute.author);
            console.log('    Source:' + res.attribute.source);
            console.log('    Time:', res.attribute.time);
            console.log('    Content', res.content);

            console.log('---------------')
        }
        console.log('-------------------------------------------------------------')
    }
};
apiInstance.getDocDisplay(dataset, opts, callback);

console.log('-------------------------------------------------------------')

----------------- Display document details --------------------
-------------------------------------------------------------
Document ID:10
    Title:D_Trump2018_8_18_1_47
    Author:D_Trump10
    Source:null
    Time: 1534582020
    Content  financial gain is a Federal Gratuity Statute Violation Bribery Statute Violation Honest Services Violation all Major Crimes because the DOJ is run by BLANK Jeff Sessions ”  Gregg Jarrett. So when does Mueller do what must be done? Probably never! @FoxNews
---------------
Document ID:11
    Title:D_Trump2018_8_18_1_46
    Author:D_Trump11
    Source:null
    Time: 1534581960
    Content “Bruce Ohr of DOJ is in legal jeopardy it’s astonishing that he’s still employed. Bruce  Nelly Ohr’s bank account is getting fatter  fatter because of the Dossier that they are both peddling. He doesn’t disclose it under Fed Regs. Using your Federal office for personal 
---------------
-------------------------------------------------------------


## Get document recommendations

In [55]:
console.log('------------- Get document recommendations -----------------')
var dataset = "trump_tweets"; // String | Dataset name.
var opts = { 
    'query': "", // String | Fulltext query, using mysql MATCH boolean query format. Example, (\"word1\" OR \"word2\") AND (\"word3\" OR \"word4\")
    //'customStopWords': "customStopWords_example", // String | List of stop words.
    'numTopics': 8, // Number | Number of topics to be extracted from the dataset.
    'numKeywords': 8, // Number | Number of keywords per topic that is extracted from the dataset.
    'numDocs': 20, // Number | Number of desired recommended docs per topic.
    //'metadataSelection': "metadataSelection_example", // String | json object of {\"metadata_field\":[\"selected_values\"]}
    //'timePeriod': "timePeriod_example", // String | Time period selection
    //'excludedDocs': "excludedDocs_example" // String | List of document IDs that should be excluded from the analysis. Example, \"docid1, docid2, ..., docidN\" 
};

var callback = function(error, data, response) {
    if (error) {
        console.error(error);
    } else {
        for (var i = 0; i < data.result.length; i++) {
            var res = data.result[i];
            console.log('Document recommendations for topic', i, ':')
            console.log('    Keywords:', res.topic);

            for (var j = 0; j < res.recommendations.length; j++) {
                var doc = res.recommendations[j];
                console.log('    Recommendation', j, ':')
                console.log('        Document ID:', doc.sourceid)
                console.log('        Title:', doc.title)
                console.log('        Author:', doc.attribute.author)
                console.log('        Source:', doc.attribute.source)
                console.log('        Time:', doc.attribute.time)
            }
    
            console.log('---------------')
        }
    }
};
apiInstance.getDocRecommendApi(dataset, opts, callback);

console.log('-------------------------------------------------------------')

------------- Get document recommendations -----------------
-------------------------------------------------------------
Document recommendations for topic 0 :
    Keywords: illegal aliens;united permission;permission illegal;enter united;aliens treated;aliens illegal;treated entered;chuck schumer
    Recommendation 0 :
        Document ID: 3657
        Title: D_Trump2018_8_15_13_18
        Author: D_Trump57
        Source: null
        Time: 1534364280
    Recommendation 1 :
        Document ID: 57
        Title: D_Trump2018_8_15_13_18
        Author: D_Trump57
        Source: null
        Time: 1534364280
    Recommendation 2 :
        Document ID: 3663
        Title: D_Trump2018_8_15_12_44
        Author: D_Trump63
        Source: null
        Time: 1534362240
    Recommendation 3 :
        Document ID: 63
        Title: D_Trump2018_8_15_12_44
        Author: D_Trump63
        Source: null
        Time: 1534362240
---------------
Document recommendations for topic 1 :
    Keywords

## Get document summary

In [56]:
console.log('------------------ Get document summary  --------------------')

var dataset = "trump_tweets"; // String | Dataset name.
var docTitle = "D_Trump2018_8_15_15_4"; // String | The title of the document to be summarized.

var opts = { 
    //'customStopWords': "customStopWords_example", // String | List of stop words
    'summaryLength': 6, // Number | The maximum number of bullet points a user wants to see in the document summary.
    'contextAmount': 0 // Number | The number of sentences surrounding key summary sentences in the documents that they come from.
};

var callback = function(error, data, response) {
    if (error) {
        console.error(error);
    } else {
        console.log('Document Summary');
        console.log('    ID:', data.result.summary.sourceid);
        console.log('    Title:', data.result.doc_title);
        console.log('    Summary:', data.result.summary.sentences);
    }
};
apiInstance.getDocSummaryApi(dataset, docTitle, opts, callback);

console.log('-------------------------------------------------------------')


------------------ Get document summary  --------------------
-------------------------------------------------------------
Document Summary
    ID: 50
    Title: D_Trump2018_8_15_15_4
    Summary: Our Country was built on Tariffs and Tariffs are now leading us to great new Trade Deals - as opposed to the horrible and unfair Trade Deals that I inherited as your President.,Other Countries should not be allowed to come in and steal the wealth of our great U.S.A. No longer!
